# AI Capabilities Identification

This notebook demonstrates how to trace from **user intent** → **AI tasks** → **required capabilities** → **intrinsic implementations**.

**This notebook demonstrates:**
- How to use LLMs to identify AI tasks from use case descriptions
- How to discover which capabilities are needed for those tasks using AtlasExplorer
- How to find which model intrinsics/adapters implement those capabilities
- An end-to-end traceability for AI system design

## 1. Setup and imports

In [1]:
import os
from rich.jupyter import print
from ai_atlas_nexus.blocks.inference import (
    RITSInferenceEngine,
    WMLInferenceEngine,
    OllamaInferenceEngine,
    VLLMInferenceEngine,
)
from ai_atlas_nexus.blocks.inference.params import (
    InferenceEngineCredentials,
    RITSInferenceEngineParams,
    WMLInferenceEngineParams,
    OllamaInferenceEngineParams,
    VLLMInferenceEngineParams,
)
from ai_atlas_nexus import AIAtlasNexus


/Users/ingevejs/Documents/workspace/ingelise/risk-atlas-nexus/src/ai_atlas_nexus/blocks/risk_explorer/explorer.py:8: UserWarning: Deprecated, Please use AtlasExplorer instead
  warnings.warn("Deprecated, Please use AtlasExplorer instead")
/Users/ingevejs/Documents/workspace/ingelise/risk-atlas-nexus/src/ai_atlas_nexus/toolkit/job_utils.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
2025-12-22 20:52:14,894 - INFO - PyTorch version 2.7.0 available.
2025-12-22 20:52:14,896 - INFO - Duckdb version 1.3.2 available.


## 2. Initialize AI Atlas Nexus

Initialize the ontology with the default capabilities, tasks, and intrinsics.

In [2]:
# Initialize AI Atlas Nexus
aan = AIAtlasNexus()

print("AI Atlas Nexus initialized:")
print(f"  {len(aan.get_all("capabilities"))} capabilities available")
print(f"  {len(aan.get_all("aitasks"))} AI tasks available")
print(f"  {len(aan.get_all("llmintrinsics") or [])} LLM intrinsics available")
print(f"  {len(aan.get_all("adapters"))} adapters available")

[2025-12-22 20:52:16:67] - INFO - AIAtlasNexus - Created AIAtlasNexus instance. Base_dir: None


AI Atlas Nexus initialized:

11 capabilities available

50 AI tasks available

9 LLM intrinsics available

19 adapters available

## 3. Configure an inference engine
**Available Inference Engines**: WML, Ollama, vLLM, RITS. Please follow the [Inference APIs](https://github.com/IBM/ai-atlas-nexus?tab=readme-ov-file#install-for-inference-apis) guide.

_Note:_ RITS is intended solely for internal IBM use and requires TUNNELALL VPN for access.

In [3]:
inference_engine = OllamaInferenceEngine(
    model_name_or_path="llama3:latest",  # Change to your preferred model
    credentials=InferenceEngineCredentials(api_url="http://localhost:11434"),
    parameters=OllamaInferenceEngineParams(
        num_predict=1000, temperature=0, repeat_penalty=1, num_ctx=8192, logprobs=False
    ),
)

print(f"\n# Inference engine configured: ")
print(f"#  Model: {inference_engine.model_name_or_path}")

[2025-12-22 20:52:16:118] - INFO - AIAtlasNexus - OLLAMA inference engine will execute requests on the server at http://localhost:11434.
[2025-12-22 20:52:16:162] - INFO - AIAtlasNexus - Created OLLAMA inference engine.


# Inference engine configured:

#  Model: llama3:latest

## 4. Example 1: RAG use case - customer support

Let's analyze a real-world RAG use case and identify what AI tasks and capabilities are needed. (source?)

In [4]:
# Define the use case
usecase = """Generate personalized, relevant responses and recommendations for 
customer support agents. The system should summarize customer claims, answer 
questions about products, and provide contextual suggestions to help agents 
resolve issues faster."""

print(f"\n# Use Case:")
print(usecase)

# Use Case:

Generate personalized, relevant responses and recommendations for 
customer support agents. The system should summarize customer claims, answer 
questions about products, and provide contextual suggestions to help agents 
resolve issues faster.

### Step 1: Identify AI tasks from use case
Use the LLM to identify which AI tasks are needed.

In [5]:
# Use AI Atlas Nexus to identify tasks
task_predictions = aan.identify_ai_tasks_from_usecases(
    usecases=[usecase],
    inference_engine=inference_engine,
)

identified_tasks = task_predictions[0].prediction

print(f"\n# Identified AI Tasks:")
for i, task in enumerate(identified_tasks, 1):
    print(f"  {i}. {task}")


Inferring with OLLAMA: 100%|██████████| 1/1 [00:10<00:00, 10.37s/it]

model='llama3:latest' created_at='2025-12-22T20:52:26.567695Z' done=True done_reason='stop' total_duration=10362765875 load_duration=5009511625 prompt_eval_count=2136 prompt_eval_duration=4509644207 eval_count=32 eval_duration=686231998 response='[\n  "Question Answering",\n  "Summarization",\n  "Text Classification",\n  "Text Generation",\n  "Text-to-Image"\n]' thinking=None context=[128006, 882, 128007, 271, 49812, 323, 3619, 279, 4823, 3770, 8649, 264, 1160, 315, 445, 11237, 3465, 323, 1202, 4096, 382, 58, 13922, 8366, 6234, 1232, 364, 15097, 55982, 518, 364, 8366, 11703, 1232, 364, 15097, 24790, 374, 279, 3465, 315, 61853, 264, 2440, 477, 538, 311, 264, 2728, 7855, 13, 1102, 649, 387, 1511, 369, 49183, 902, 3290, 264, 1217, 374, 7231, 477, 279, 20356, 315, 264, 5224, 11, 439, 1664, 439, 25607, 264, 19114, 3238, 2186, 5473, 8366, 6234, 1232, 364, 15097, 4791, 6830, 3310, 518, 364, 8366, 11703, 1232, 364, 15097, 4791, 6830, 3310, 374, 264, 3070, 315, 9256, 304, 902, 279, 1988, 374, 4

# Identified AI Tasks:

1. Question Answering

2. Summarization

3. Text Classification

4. Text Generation

5. Text-to-Image

### Step 2: Explore Available Capabilities

Let's see what capabilities are defined in the IBM AI Capabilities Framework.

In [6]:
# Get all capabilities, limited to IBM AI Capabilities Framework
capabilities = aan.get_all("capabilities", taxonomy="ibm-ai-capabilities")
print(f"\n# IBM AI Capabilities Framework: {len(capabilities)} capabilities\n")

# Group by capability group
for cap in sorted(capabilities, key=lambda x: x.isPartOf):
    print(f"{cap.name}")
    print(f"  ID: {cap.id}")
    print(f"  Group: {cap.isPartOf}")
    if hasattr(cap, 'description') and cap.description:
        # Truncate long descriptions
        desc = cap.description[:100] + "..." if len(cap.description) > 100 else cap.description
        print(f"  Description: {desc}")
    print()

# IBM AI Capabilities Framework: 11 capabilities

Domain Knowledge

ID: ibm-cap-domain-knowledge

Group: ibm-cap-domain-knowledge-and-learning

Description: Possessing and applying specialized knowledge across technical, scientific, and professional 
domains...

...

Description: The ability to understand cause-effect relationships, identify causal chains, and reason about 
inter...

Mathematical Reasoning

ID: ibm-cap-mathematical-reasoning

Group: ibm-cap-group-mathematical

Description: The ability to solve mathematical problems, perform numerical calculations, and apply mathematical 
c...

### Step 3: Retrieve mapping of tasks to capabilities

Now we can query which capabilities are required for each task.

In [7]:
# Example: Get capabilities for Question Answering task
# Query instances with mutiple kwargs
qa_capabilities = aan.query(
    class_name="capabilities", 
    requiredByTask="question-answering", 
    isDefinedByTaxonomy="ibm-ai-capabilities"
)

print("Question Answering task requires these capabilities:\n")
for cap in qa_capabilities:
    print(f"  • {cap.name}")
    print(f"    ID: {cap.id}")
    if hasattr(cap, 'description') and cap.description:
        desc = cap.description[:120] + "..." if len(cap.description) > 120 else cap.description
        print(f"    Description: {desc}")
    print()

print(f"\n✓ Found {len(qa_capabilities)} capabilities required for Question Answering")

Question Answering task requires these capabilities:

• Reading Comprehension

ID: ibm-cap-reading-comprehension

Description: The ability to understand written text, extract key information, and answer questions about the 
content. This is a found...

• Contextual Understanding

...

Description: Possessing and applying specialized knowledge across technical, scientific, and professional 
domains including medicine,...

• Factual Accuracy

ID: ibm-cap-factual-accuracy

Description: The ability to produce factually correct information and avoid hallucinations, confabulations, or 
false statements. Esse...

✓ Found 5 capabilities required for Question Answering

### Step 4: Find intrinsics that implement capabilities

For each capability, we can find which intrinsics and adapters implement it.

In [8]:
# Example: Get intrinsics for Reading Comprehension capability
intrinsics = aan.query(class_name="adapters",
    implementsCapability="ibm-cap-reading-comprehension",
)

print("Reading Comprehension is implemented by:\n")
for intr in intrinsics:
    print(f"  • {intr.name}")
    print(f"    ID: {intr.id}")
    print()

print(f"\n✓ Found {len(intrinsics)} implementations for Reading Comprehension")

Reading Comprehension is implemented by:

• Granite 3.3 8b Instruct - Query Rewrite

ID: ibm-factuality-adapter-granite-3.3-8b-instruct-lora-query-rewrite

• Granite 3.3 8b Instruct - Context Relevance

ID: ibm-factuality-adapter-granite-3.3-8b-instruct-lora-context-relevance

• Granite 3.3 8b Instruct - Answerability Determination

ID: ibm-factuality-adapter-granite-3.3-8b-instruct-lora-answerability-determination

• Granite 3.3 Instruct - Passage Reranking

ID: ibm-factuality-adapter-granite-3.3-instruct-lora-passage-reranking

✓ Found 4 implementations for Reading Comprehension